In [ ]:
import numpy as np
import pyvista as pv

In pyvista you require to triangulate the mesh before calculating the mesh area. Currently, there is no funcionality to calculate the area of an individual mesh cell face (not that I can easily find).

To circumvent this, this clumsy solution simply deconstructs the pyvista.PolyData to then form a PolyData mesh for each cell face, triangulate the mesh containing a single cell face, then calculate the mesh area which should be eqyivalent to the original mesh cell face area.

The list of cells areas are kept in order, and then reattached to the original mesh as a cell array.

In [ ]:
resolution = 1668

In [ ]:
mesh = pv.read(f"pdata_C{resolution}.vtk")

In [ ]:
points = mesh.points

In [ ]:
faces = mesh.faces.reshape(-1, 5)

In [ ]:
areas = []

In [ ]:
for fidx in range(faces.shape[0]):
    face = faces[fidx]
    pdata = pv.PolyData(points, np.array(face))
    areas.append(pdata.triangulate().area)

In [ ]:
areas = np.array(areas)

In [ ]:
print(f"mesh area difference: {mesh.triangulate().area - areas.sum()}")

In [ ]:
mesh["area"] = areas

In [ ]:
mesh.save(f"pdata_area_C{resolution}.vtk")

In [ ]:
!ls -l pdata_area*

----

In [ ]:
mesh = pv.read(f"pdata_C{resolution}.vtk")

In [ ]:
mesh

In [ ]:
amesh = pv.read(f"pdata_area_C{resolution}.vtk")

In [ ]:
amesh

In [ ]:
tmesh = mesh.triangulate()

In [ ]:
tmesh

In [ ]:
area = amesh["area"]

In [ ]:
np.min(area), np.max(area), np.median(area), np.mean(area), np.std(area)

In [ ]:
tarea = tmesh.area
carea = np.sum(area)
diff = tarea - carea
tarea, carea, diff

In [ ]:
actual = 4*np.pi

In [ ]:
actual - tarea, ((actual - tarea) / actual) * 100, actual - carea, ((actual - carea) / actual) * 100